# DATA ACQUISITION

Chúng ta sẽ tải bộ dữ liệu The Look trên Kaggle.
Đây là bộ dữ liệu synthetic từ Google về một hệ thống ecommerce đầy đủ về khách hàng, đơn hàng, sản phẩm, kho hàng, vv

In [1]:
import kaggle

In [2]:
kaggle.api.dataset_download_files('daichiuchigashima/thelook-ecommerce', path='../data', unzip=True)

print("Download complete!")

Dataset URL: https://www.kaggle.com/datasets/daichiuchigashima/thelook-ecommerce
Download complete!


# EXPLORE

Ở đây chúng ta sẽ load và tổng hợp data từ nhiều file csv khác nhau để tạo training data.

In [1]:
import duckdb
import glob
import pandas as pd

In [2]:
# Tạo một dict để lưu tất cả đường dẫn đến data csv. Việc này sẽ giúp tiết kiệm thời gian và làm SQL query trông đẹp hơn

tables = {}

for csv_file in glob.glob("../data/*csv"):
    tables[csv_file.split(".")[-2]] = f"'{csv_file}'" # Thêm dấu ' vì duckdb cần wrap đường dẫn bởi symbol này.

tables

{'orders': "'../data/thelook_ecommerce.orders.csv'",
 'order_items': "'../data/thelook_ecommerce.order_items.csv'",
 'events': "'../data/thelook_ecommerce.events.csv'",
 'products': "'../data/thelook_ecommerce.products.csv'",
 'inventory_items': "'../data/thelook_ecommerce.inventory_items.csv'",
 'users': "'../data/thelook_ecommerce.users.csv'",
 'distribution_centers': "'../data/thelook_ecommerce.distribution_centers.csv'"}

In [3]:
# Dùng DuckDB để query CSV như một relation db

con = duckdb.connect() # Create a connection of an in-memory database
query = f"""
WITH sales_data as (
    SELECT
        CAST(oi.created_at AS DATE) as sale_date,
        oi.product_id,
        p.category,
        oi.sale_price,
        ii.cost,
        u.traffic_source
    
    FROM {tables['order_items']} oi
    JOIN {tables['inventory_items']} ii ON oi.inventory_item_id = ii.id
    JOIN {tables['products']} p ON oi.product_id = p.id
    JOIN {tables['users']} u ON oi.user_id = u.id

    WHERE oi.status NOT IN ('Cancelled', 'Returned')
)

SELECT
    sale_date,
    category,
    traffic_source,
    
    -- Tính giá trung bình
    AVG(sale_price) as avg_price,

    -- Tính cost trung bình
    AVG(cost) as cost,

    -- Tính số lượng bán ra trung bình
    COUNT(*) as qty_sold,

    -- Tính ngày trong tuần (0 = Chủ Nhật)
    DAYOFWEEK(sale_date) as day_of_week
FROM sales_data
GROUP BY sale_date, category, traffic_source
ORDER BY sale_date
"""
df = con.execute(query).df()
df

,sale_date,category,traffic_source,avg_price,cost,qty_sold,day_of_week
0,2019-01-12,Plus,Search,10.000000,5.230000,1,6
1,2019-01-12,Fashion Hoodies & Sweatshirts,Search,57.000000,32.319000,1,6
2,2019-01-13,Pants & Capris,Facebook,38.560001,18.663041,1,0
3,2019-01-13,Shorts,Facebook,77.000000,38.115000,1,0
4,2019-01-16,Fashion Hoodies & Sweatshirts,Search,49.500000,24.849000,1,3
...,...,...,...,...,...,...,...
48641,2022-06-12,Tops & Tees,Organic,17.950001,10.213550,1,0
48642,2022-06-12,Swim,Search,31.013334,16.147334,3,0
48643,2022-06-12,Jeans,Organic,39.990002,23.034241,1,0
48644,2022-06-12,Suits & Sport Coats,Display,98.000000,40.474000,1,0


In [7]:
df.to_csv("../data/train_data.csv")
print(f"Tổng hợp được số lượng training data: {len(df):,d}")

Tổng hợp được số lượng training data: 48,646
